In [1]:
import os
from PIL import Image
from transformers import DonutProcessor, VisionEncoderDecoderModel

# Load processor and model
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")


KeyboardInterrupt: 

In [ ]:
def process_invoice_image(image_path, task_prompt="<s_invoices>", max_length=512):
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

    outputs = model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=max_length
    )
    result = processor.batch_decode(outputs, skip_special_tokens=True)[0]
    return result


In [ ]:
# Set your folder path here
image_folder = "/mnt/c/Users/Legion/Documents/jimmy tran/Automated-invoice-processing-system/dataset/converted_pngs"

# Get all PNG files
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(".png")]

# Run through each image
results = {}

for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    print(f"Processing: {image_file}")
    prediction = process_invoice_image(image_path)
    print(f"→ Prediction:\n{prediction}\n")
    results[image_file] = prediction


In [ ]:
import json

output_path = "donut_predictions.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"✅ Saved predictions to: {output_path}")
